# ÖDEV 1: PCA yardımı ile Classification,

Bu ödevde "Credit Risk Prediction" veri setini kullanacağız. Amacımız, verinin boyut sayısını düşürerek olabildiğince yüksek accuracy değerini alabilmek. Aşağıda verinin okunma ve temizlenme kısmını hazırlayıp vereceğim. Devamında ise yapmanız gerekenler:

1. PCA kullanarak verinin boyutunu düşürmek
    * Önce explained varience ratio değerini inceleyerek veriyi kaç boyuta düşürebileceğini kontrol et.
    * Daha sonra farklı boyutlarda denemeler yaparak boyutu düşürülmüş verileri elde et.
2. Classification modellerini dene
    * Logistic Regression
    * Random Forest
    * ve eğer istersen herhangi bir modelle daha

İsteğe bağlı olarak, verinin boyutunu düşürmek için diğer yöntemleri de kullanıp en yüksek accuracy değerini almayı deneyebilirsin.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df: pd.DataFrame = pd.read_csv('./credit_risk_dataset.csv')

In [3]:
print(df.isnull().sum())

person_age                       0
person_income                    0
person_home_ownership            0
person_emp_length              895
loan_intent                      0
loan_grade                       0
loan_amnt                        0
loan_int_rate                 3116
loan_status                      0
loan_percent_income              0
cb_person_default_on_file        0
cb_person_cred_hist_length       0
dtype: int64


In [4]:
# Null değerleri sütun ortalaması ile dolduruyoruz
df["person_emp_length"].fillna(df["person_emp_length"].median(), inplace=True)
df["loan_int_rate"].fillna(df["loan_int_rate"].median(), inplace=True)

In [5]:
df.duplicated().sum()

165

In [6]:
df.drop_duplicates(inplace=True)

In [7]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
person_age,32416.0,27.747008,6.354100,20.00,23.00,26.00,30.00,144.00
person_income,32416.0,66091.640826,62015.580269,4000.00,38542.00,55000.00,79218.00,6000000.00
person_emp_length,32416.0,4.768880,4.090411,0.00,2.00,4.00,7.00,123.00
loan_amnt,32416.0,9593.845632,6322.730241,500.00,5000.00,8000.00,12250.00,35000.00
loan_int_rate,32416.0,11.014662,3.083050,5.42,8.49,10.99,13.11,23.22
loan_status,32416.0,0.218688,0.413363,0.00,0.00,0.00,0.00,1.00
loan_percent_income,32416.0,0.170250,0.106812,0.00,0.09,0.15,0.23,0.83
cb_person_cred_hist_length,32416.0,5.811297,4.059030,2.00,3.00,4.00,8.00,30.00


In [8]:
# Outlier temizliği
df = df[df['person_age']<=100]
df = df[df['person_emp_length'] <= 60]
df = df[df['person_income']<=4e6]

In [9]:
# Kategorik verileri alıyoruz ve one hot encoding haline getiriyoruz
cat_cols = pd.DataFrame(df[df.select_dtypes(include=['object']).columns])
cat_cols.columns

Index(['person_home_ownership', 'loan_intent', 'loan_grade',
       'cb_person_default_on_file'],
      dtype='object')

In [10]:
encoded_cat_cols = pd.get_dummies(cat_cols)
df.drop(df.select_dtypes(include=['object']).columns, axis=1,inplace=True)
df = pd.concat([df,encoded_cat_cols], axis=1)

In [11]:
X = df.drop('loan_status', axis=1).values
y = df['loan_status'].values

In [12]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(1, test_size=0.1)
train_idx, test_idx = next(split.split(X, y))
train_x = X[train_idx]
test_x = X[test_idx]

train_y = y[train_idx]
test_y = y[test_idx]

# PCA (Principal Component Analysis)

PCA, popüler bir boyut azaltma yöntemidir. Temel olarak bir projeksiyon işlemi yapar; ilk olarak veriye en uygun uzayı tanımlar ve istenen boyuta indirgemek için temel bileşenleri kullanarak veriyi alt boyuta yansıtır.

Temel bileşenler, SVD (Singular Value Decomposition) adı verilen bir matris çarpımından elde edilir. Veri, 3 matrisin çarpımı şeklinde ifade edilir ve bu matrislerden biri temel bileşenleri içeren V matrisinin transpozudur. Düşürülmek istenen boyut kadar temel bileşeni içeren matrisle veri çarpılır, bu şekilde yeni veri elde edilir.

**PCA, bir orijin etrafında dağılmış şekilde olan veriye uygulanır, veri bu halde değilse bu hale getirilmedilir. Sklearn PCA sınıfı bu işlemi otomatik yapar.**

In [13]:
from sklearn.decomposition import PCA

In [14]:
pca = PCA(n_components=2)
X_train_2d = pca.fit_transform(train_x)
X_test_2d = pca.transform(test_x)

In [15]:
pca.components_.T

array([[ 1.63902202e-05, -9.00534957e-06],
       [ 9.99267509e-01,  3.82680580e-02],
       [ 1.19151653e-05, -4.37782299e-05],
       [ 3.82680574e-02, -9.99267506e-01],
       [-9.83842189e-08, -7.47661530e-05],
       [-5.88426323e-07, -1.25056071e-05],
       [ 9.29050695e-06, -3.08326839e-06],
       [ 2.19633578e-06, -4.92195564e-06],
       [ 1.25914405e-08, -4.97205338e-08],
       [-2.25842169e-07,  4.99321667e-07],
       [-1.98308505e-06,  4.47235450e-06],
       [ 3.68964170e-08,  1.90973272e-07],
       [-1.25359915e-07,  3.96658192e-07],
       [ 3.13750202e-07, -1.54133366e-06],
       [-3.18772434e-07,  5.63387055e-07],
       [ 4.74983886e-08,  9.06117341e-08],
       [ 4.59873404e-08,  2.99703405e-07],
       [ 7.01561027e-08,  9.62740416e-06],
       [ 8.34773453e-08, -3.18260241e-06],
       [-1.54275283e-07,  1.77469101e-06],
       [-9.87241709e-08, -4.20600797e-06],
       [ 6.42868084e-08, -2.55887540e-06],
       [ 2.68023481e-08, -1.04654621e-06],
       [ 8.

In [16]:
pca.explained_variance_ratio_

array([0.98722002, 0.01277995])

İlk component verinin varyansının 98%'sini içeriyormuş.

# Classification

## GradientBoosting

In [19]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, mean_squared_error

gbc = GradientBoostingClassifier(max_depth=2, min_samples_split=3, n_estimators=150)
gbc.fit(X_train_2d, train_y)

errors = [mean_squared_error(test_y, y_pred) for y_pred in gbc.staged_predict(X_test_2d)]
acc = [accuracy_score(test_y, y_pred) for y_pred in gbc.staged_predict(X_test_2d)]
best_n_estimators = np.argmax(acc)

gbc = GradientBoostingClassifier(max_depth=3, n_estimators=best_n_estimators)
gbc.fit(X_train_2d,train_y)
y_pred = gbc.predict(X_test_2d)
accuracy_gbc = accuracy_score(test_y, y_pred)*100

print("Gradient Boosting Classifier Accuracy: ",accuracy_gbc,"%")

Gradient Boosting Classifier Accuracy:  83.64702252391237 %


## RandomForests

In [20]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=60, bootstrap=True)
rfc.fit(X_train_2d, train_y)
y_pred = rfc.predict(X_test_2d)
accuracy_rfc = accuracy_score(test_y, y_pred)*100
print("Ranfom Forests accuracy on test set: ",accuracy_rfc,"%")

Ranfom Forests accuracy on test set:  83.52360382597963 %


## Neural Network

In [21]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_nn = scaler.fit_transform(X_train_2d)
X_test_nn = scaler.transform(X_test_2d)

input_size = X_train_nn.shape[1]

model = Sequential()
model.add(Dense(units=input_size, input_dim=X_train_nn.shape[1], activation="relu"))
model.add(Dense(units=input_size, activation="relu"))
model.add(Dense(units=input_size, activation="relu"))
model.add(Dense(units=input_size, activation="relu"))
model.add(Dense(units=input_size, activation="relu"))
model.add(Dense(units=input_size/2, activation="relu"))
model.add(Dense(units=1, activation="sigmoid"))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'] )
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 2)                 6         
                                                                 
 dense_1 (Dense)             (None, 2)                 6         
                                                                 
 dense_2 (Dense)             (None, 2)                 6         
                                                                 
 dense_3 (Dense)             (None, 2)                 6         
                                                                 
 dense_4 (Dense)             (None, 2)                 6         
                                                                 
 dense_5 (Dense)             (None, 1)                 3         
                                                                 
 dense_6 (Dense)             (None, 1)                 2

In [22]:
history = model.fit(X_train_nn, train_y, epochs=100, verbose=10, validation_split=0.2)

Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

In [23]:
print("Accuracy on validation set: ",history.history["val_accuracy"][-1]*100,"%")

Accuracy on validation set:  77.52828001976013 %


In [24]:
y_pred_nn = model.predict(X_test_nn)
accuracy = np.dot(history.history["accuracy"],100)
loss = history.history['loss']

score = model.evaluate(X_test_nn, test_y, steps=5)
accuracy_nn = score[1]*100
print()
print("Accuracy on test set: ",accuracy_nn,"%")

5/5 [==============================] - 0s 2ms/step - loss: 0.5253 - accuracy: 0.7812

Accuracy on test set:  78.12403440475464 %


## Support Vector Machines

In [25]:
from sklearn.svm import SVC

svc = SVC(kernel="rbf")
svc.fit(X_train_2d, train_y)
y_pred = svc.predict(X_test_2d)
accuracy_svc = accuracy_score(test_y, y_pred)*100
print("SVC Accuracy on test set: ",accuracy_svc,"%")

SVC Accuracy on test set:  81.57975933353903 %


### Comparision

In [26]:
compare = pd.DataFrame(index=["Random Forests","Gradient Boosting Classifier","Neural Network","Support Vector Classifier"],
                       columns=["Accuracy"],
                      data=[accuracy_rfc,accuracy_gbc,accuracy_nn,accuracy_svc])
compare

,Accuracy
Random Forests,83.523604
Gradient Boosting Classifier,83.647023
Neural Network,78.124034
Support Vector Classifier,81.579759
